In [1]:
import glob
import pandas as pd
import urllib
import requests
import xml.etree.ElementTree as ET
import time
import numpy as np
from itertools import zip_longest
import re

In [2]:
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [3]:
df = pd.read_csv('/home/riddleta/ac_knowl/output/file_index.csv')

/home/riddleta/ac_knowl/ac_knowl/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,file,pmcid,paper_number
0,/home/riddleta/ac_knowl/output/full_texts/pape...,3972225,0
1,/home/riddleta/ac_knowl/output/full_texts/pape...,3972226,1
2,/home/riddleta/ac_knowl/output/full_texts/pape...,3972227,2
3,/home/riddleta/ac_knowl/output/full_texts/pape...,3972228,3
4,/home/riddleta/ac_knowl/output/full_texts/pape...,3972229,4


In [38]:
with open('api_key.txt', 'r') as infile:
    line = infile.readlines()
    api_key = line[0][10:]

In [7]:
mesh_labs = []
mesh_ids = []
grant_ids = []
grant_agency = []
journal_out = []
pmid_out = []
auths_out = []
pmcid_out = []
total_papes = 0
for i in grouper(df.pmcid.tolist(), 200):
    try:
        i = list(map(str, i)) #ensure strings
        pth = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pmc&id=' + str(','.join(i)) + '&retmode=json&api_key=' + str(api_key)
        r = requests.get(pth)
        r_out = r.json()
        jr=[] #store local group results
        pmr=[]
        ar=[]
        pmcr=[]
        for j in r_out['result']['uids']:
            jr.append(r_out['result'][str(j)]['fulljournalname'])
            pmr.append([art_id['value'] for art_id in r_out['result'][str(j)]['articleids'] if art_id['idtype']=='pmid'][0])
            ar.append(r_out['result'][str(j)]['authors'])
            pmcr.append(j)


        journal_out.extend(jr) #build up lists for writing
        pmid_out.extend(pmr)
        auths_out.extend(ar)
        pmcid_out.extend(pmcr)

        out_df = pd.DataFrame({'journal':journal_out,
                          'pmid':pmid_out,
                          'auths':auths_out,
                          'pmcid':pmcid_out})
        if out_df.shape[0]>100000:
            total_papes = total_papes + out_df.shape[0]
            fn = 'grant_id_'
            out_df.to_csv('/home/riddleta/ac_knowl/output/grant_identifiers/'+fn+str(total_papes)+'.csv', index=False)
            journal_out = []
            pmid_out = []
            auths_out = []
            pmcid_out = []

        time.sleep(.11)
    except Exception as e:
        print(str(i[0])+'*******'+str(i[199]))
        print(e)
        time.sleep(.11)


5846053*******5846376
'fulljournalname'
6450715*******6456894
'fulljournalname'
5390906*******5391162
'fulljournalname'
4775292*******4776336
'fulljournalname'
5929106*******5931393
'fulljournalname'
5822532*******5822787
'fulljournalname'
5819009*******4370912
Expecting value: line 1 column 1 (char 0)
4370913*******4371426
Expecting value: line 1 column 1 (char 0)
4107366*******4108450
'fulljournalname'
4015299*******4015605
'fulljournalname'
5131257*******5133195
'fulljournalname'
4156544*******4157118
'fulljournalname'
5727783*******5728061
'fulljournalname'
4643749*******4644007
'fulljournalname'
5669235*******5669591
'fulljournalname'
6373394*******6373698
'fulljournalname'
4023268*******4023626
'fulljournalname'
4020188*******4020463
'fulljournalname'
5641422*******5642984
'fulljournalname'
6458428*******6458657
'fulljournalname'


In [16]:
total_papes = total_papes + out_df.shape[0]
fn = 'grant_id_'
out_df.to_csv('/home/riddleta/ac_knowl/output/grant_identifiers/'+fn+str(total_papes)+'.csv', index=False)

In [17]:
total_papes

2544923

In [15]:
out_df.shape

(39924, 4)

In [18]:
li = []
all_files = glob.glob('/home/riddleta/ac_knowl/output/grant_identifiers/*')
for filename in all_files:
    tempdat = pd.read_csv(filename, index_col=None, header=0)
    li.append(tempdat)

frame = pd.concat(li, axis=0, ignore_index=True)

In [19]:
frame.shape

(2544923, 4)

In [21]:
stragglers = df[~df.pmcid.isin(frame.pmcid)]
stragglers.shape

(316144, 3)

In [22]:
mesh_labs = []
mesh_ids = []
grant_ids = []
grant_agency = []
journal_out = []
pmid_out = []
auths_out = []
pmcid_out = []
#total_papes = 2504999
for i in grouper(stragglers.pmcid.tolist(), 200):
    try:
        i = list(map(str, i)) #ensure strings
        pth = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pmc&id=' + str(','.join(i)) + '&retmode=json&api_key=' + str(api_key)
        r = requests.get(pth)
        r_out = r.json()
        jr=[] #store local group results
        pmr=[]
        ar=[]
        pmcr=[]
        for j in r_out['result']['uids']:
            try:
                jr.append(r_out['result'][str(j)]['fulljournalname'])
                pmr.append([art_id['value'] for art_id in r_out['result'][str(j)]['articleids'] if art_id['idtype']=='pmid'][0])
                ar.append(r_out['result'][str(j)]['authors'])
                pmcr.append(j)
            except:
                jr.append(np.nan)
                pmr.append(np.nan)
                ar.append(np.nan)
                pmcr.append(j)


        journal_out.extend(jr) #build up lists for writing
        pmid_out.extend(pmr)
        auths_out.extend(ar)
        pmcid_out.extend(pmcr)

        out_df = pd.DataFrame({'journal':journal_out,
                          'pmid':pmid_out,
                          'auths':auths_out,
                          'pmcid':pmcid_out})
        if out_df.shape[0]>100000:
            total_papes = total_papes + out_df.shape[0]
            fn = 'grant_id_'
            out_df.to_csv('/home/riddleta/ac_knowl/output/grant_identifiers/'+fn+str(total_papes)+'.csv', index=False)
            journal_out = []
            pmid_out = []
            auths_out = []
            pmcid_out = []

        time.sleep(.11)
    except Exception as e:
        print(str(i[0])+'*******'+str(i[199]))
        print(e)
        time.sleep(.11)

5819607*******4371060
('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))


In [24]:
total_papes = total_papes + out_df.shape[0]
fn = 'grant_id_'
out_df.to_csv('/home/riddleta/ac_knowl/output/grant_identifiers/'+fn+str(total_papes)+'.csv', index=False)

In [46]:
li = []
all_files = glob.glob('/home/riddleta/ac_knowl/output/grant_identifiers/*')
for filename in all_files:
    tempdat = pd.read_csv(filename, index_col=None, header=0)
    li.append(tempdat)

frame = pd.concat(li, axis=0, ignore_index=True)
frame = frame.drop_duplicates()
frame['pmcid'] = frame.pmcid.astype('str')

In [47]:
frame.shape

(2599324, 4)

In [48]:
df['pmcid'] = df.pmcid.astype('str')
df.shape

(2599524, 3)

In [49]:
stragglers = df[~df.pmcid.isin(frame.pmcid)]
stragglers.shape

(201, 3)

In [52]:
journal_out = []
pmid_out = []
auths_out = []
pmcid_out = []
for i in stragglers.pmcid.tolist():
    try:
        pth = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pmc&id=' + str(i) + '&retmode=json&api_key=' + str(api_key)
        r = requests.get(pth)
        r_out = r.json()
        journal_out.append(r_out['result'][str(i)]['fulljournalname'])
        pmid_out.append([art_id['value'] for art_id in r_out['result'][str(i)]['articleids'] if art_id['idtype']=='pmid'][0])
        auths_out.append(r_out['result'][str(i)]['authors'])
        pmcid_out.append(i)

        time.sleep(.11)
    except Exception as e:
        print(str(i))
        print(e)
        journal_out.append(np.nan)
        pmid_out.append(np.nan)
        auths_out.append(np.nan)
        pmcid_out.append(i)
        time.sleep(.11)


out_df = pd.DataFrame({'journal':journal_out,
                  'pmid':pmid_out,
                  'auths':auths_out,
                  'pmcid':pmcid_out})

unknown
'unknown'


In [56]:
total_papes = total_papes + out_df.shape[0]
fn = 'grant_id_'
out_df.to_csv('/home/riddleta/ac_knowl/output/grant_identifiers/'+fn+str(total_papes)+'.csv', index=False)

## Hit the grant API
pull the mesh labels, ids, grant ids, and grant agency for each paper

In [15]:
li = []
all_files = glob.glob('/home/riddleta/ac_knowl/output/pmid_pmcid_link/*')
for filename in all_files:
    tempdat = pd.read_csv(filename, index_col=None, header=0)
    li.append(tempdat)

frame = pd.concat(li, axis=0, ignore_index=True)
frame['pmcid'] = frame.pmcid.astype('str')
#frame['pmid'] = frame.pmid.astype('int')
#frame['pmid'] = frame.pmid.astype('str')
frame = frame.drop_duplicates()


In [17]:
frame['pmid2'] = frame.pmid.fillna(0).astype('int')
frame.pmid2.head()

0    28674422
1    28674426
2    28674452
3    28674872
4    28674848
Name: pmid2, dtype: int64

In [42]:
frame.shape

(2599525, 5)

In [39]:
#initialize some lists to hold data at the paper level
paper_level_meshlabs = []
paper_level_meshids = []
paper_level_grant_ids = []
paper_level_grant_agency = []
paper_level_kw = []
pmid_out = []
#keep track of how many papers
total_papes = 0
fn = 'grant_id_'
#for each group of 200 papers
for i in grouper(frame.pmid2.tolist(), 200):
    try:
        i = list(map(str, i)) #ensure strings
        pth = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=' + str(','.join(i)) + '&retmode=xml&api_key=' + str(api_key)
        #get the data
        r = requests.get(pth)
        root = ET.fromstring(r.content)
        # for each paper in the group of 200
        for paper in root.iter('PubmedArticle'):
            #initialize some lists to hold the variable-length content from a paper
            tmp_mesh_labs = []
            tmp_mesh_ids = []
            tmp_grant_ids = []
            tmp_grant_agency = []
            tmp_kw = []
            if paper.find('.//MeshHeading'):
                for mesh in paper.iter('MeshHeading'):
                    tmp_mesh_labs.append([j.text for j in mesh])
                    tmp_mesh_ids.append([j.attrib['UI'] for j in mesh])
            else:
                tmp_mesh_labs.append(np.nan)
                tmp_mesh_ids.append(np.nan)
            if paper.find('.//KeywordList'):
                for keyword in paper.iter('KeywordList'):
                    tmp_kw.append([j.text for j in keyword])
            else:
                tmp_kw.append(np.nan)
            if paper.find('.//Grant'):
                for grant in paper.iter('Grant'):
                    tmp_grant_ids.append([j.text for j in grant if j.tag=='GrantID'])
                    tmp_grant_agency.append([j.text for j in grant if j.tag=='Agency'])
            else:
                tmp_grant_ids.append(np.nan)
                tmp_grant_agency.append(np.nan)

            paper_level_meshlabs.append(tmp_mesh_labs)
            paper_level_meshids.append(tmp_mesh_ids)
            paper_level_grant_ids.append(tmp_grant_ids)
            paper_level_grant_agency.append(tmp_grant_agency)
            paper_level_kw.append(tmp_kw)
            pmid_out.append(paper[0][0].text)
    
        out_df = pd.DataFrame({'mesh':paper_level_meshlabs,
                               'mesh_id':paper_level_meshids,
                               'grant_ids':paper_level_grant_ids,
                               'grant_agency':paper_level_grant_agency,
                               'keyword':paper_level_kw,
                               'pmid':pmid_out})
        
        if out_df.shape[0]>100000:
            total_papes = total_papes + out_df.shape[0]
            out_df.to_csv('/home/riddleta/ac_knowl/output/grant_identifiers/'+fn+str(total_papes)+'.csv', index=False)
            paper_level_meshlabs = []
            paper_level_meshids = []
            paper_level_grant_ids = []
            paper_level_grant_agency = []
            paper_level_kw = []
            pmid_out = []
        
        time.sleep(.11)
            
    except Exception as e:
        print(str(i[0])+'*******'+str(i[199]))
        print(e)
        time.sleep(.11)

23590562*******23504439
('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))
30950790*******23256870
undefined entity &nbsp;: line 28, column 94


In [45]:
total_papes

2502692

In [46]:
total_papes = total_papes + out_df.shape[0]
out_df.to_csv('/home/riddleta/ac_knowl/output/grant_identifiers/'+fn+str(total_papes)+'.csv', index=False)

In [47]:
li = []
all_files = glob.glob('/home/riddleta/ac_knowl/output/grant_identifiers/*')
for filename in all_files:
    tempdat = pd.read_csv(filename, index_col=None, header=0)
    li.append(tempdat)

done_dat = pd.concat(li, axis=0, ignore_index=True)

In [48]:
stragglers = frame[~frame.pmid2.isin(done_dat.pmid)]
stragglers.shape

(75741, 5)

In [49]:
#initialize some lists to hold data at the paper level
paper_level_meshlabs = []
paper_level_meshids = []
paper_level_grant_ids = []
paper_level_grant_agency = []
paper_level_kw = []
pmid_out = []
#keep track of how many papers
total_papes = 0
fn = 'grant_id_'
#for each group of 200 papers
for i in grouper(stragglers.pmid2.tolist(), 200):
    try:
        i = list(map(str, i)) #ensure strings
        pth = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=' + str(','.join(i)) + '&retmode=xml&api_key=' + str(api_key)
        #get the data
        r = requests.get(pth)
        root = ET.fromstring(r.content)
        # for each paper in the group of 200
        for paper in root.iter('PubmedArticle'):
            #initialize some lists to hold the variable-length content from a paper
            tmp_mesh_labs = []
            tmp_mesh_ids = []
            tmp_grant_ids = []
            tmp_grant_agency = []
            tmp_kw = []
            if paper.find('.//MeshHeading'):
                for mesh in paper.iter('MeshHeading'):
                    tmp_mesh_labs.append([j.text for j in mesh])
                    tmp_mesh_ids.append([j.attrib['UI'] for j in mesh])
            else:
                tmp_mesh_labs.append(np.nan)
                tmp_mesh_ids.append(np.nan)
            if paper.find('.//KeywordList'):
                for keyword in paper.iter('KeywordList'):
                    tmp_kw.append([j.text for j in keyword])
            else:
                tmp_kw.append(np.nan)
            if paper.find('.//Grant'):
                for grant in paper.iter('Grant'):
                    tmp_grant_ids.append([j.text for j in grant if j.tag=='GrantID'])
                    tmp_grant_agency.append([j.text for j in grant if j.tag=='Agency'])
            else:
                tmp_grant_ids.append(np.nan)
                tmp_grant_agency.append(np.nan)

            paper_level_meshlabs.append(tmp_mesh_labs)
            paper_level_meshids.append(tmp_mesh_ids)
            paper_level_grant_ids.append(tmp_grant_ids)
            paper_level_grant_agency.append(tmp_grant_agency)
            paper_level_kw.append(tmp_kw)
            pmid_out.append(paper[0][0].text)
    
        out_df = pd.DataFrame({'mesh':paper_level_meshlabs,
                               'mesh_id':paper_level_meshids,
                               'grant_ids':paper_level_grant_ids,
                               'grant_agency':paper_level_grant_agency,
                               'keyword':paper_level_kw,
                               'pmid':pmid_out})
        time.sleep(.11)
            
    except Exception as e:
        print(str(i[0])+'*******'+str(i[199]))
        print(e)
        time.sleep(.11)

In [54]:
#Lots of these are just zero. missing data i guess
pd.Series(stragglers.pmid2==0).value_counts()

True     75348
False      393
Name: pmid2, dtype: int64

In [55]:
total_papes = total_papes + out_df.shape[0]
out_df.to_csv('/home/riddleta/ac_knowl/output/grant_identifiers/'+fn+str(total_papes)+'.csv', index=False)

In [3]:
li = []
all_files = glob.glob('/home/riddleta/ac_knowl/output/grant_identifiers/*')
for filename in all_files:
    tempdat = pd.read_csv(filename, index_col=None, header=0)
    li.append(tempdat)

done_dat = pd.concat(li, axis=0, ignore_index=True)

# reformat grant-pmid pairing
Now, I want a file that has a grant id, the grant agency, and pmid

In [4]:
grant_funded = done_dat[done_dat.grant_ids!='[nan]']
grant_funded.shape

(921814, 6)

In [5]:
grant_funded.head()

,mesh,mesh_id,grant_ids,grant_agency,keyword,pmid
3,[nan],[nan],"[['RC2 GM092561'], ['T32 GM083864']]","[['NIGMS NIH HHS'], ['NIGMS NIH HHS']]","[['gas chromatography', 'high performance liqu...",25789275
6,[nan],[nan],"[['R01 DA007606'], ['R01 DA035499']]","[['NIDA NIH HHS'], ['NIDA NIH HHS']]","[['blood-brain barrier', 'glutamate', 'methamp...",25788874
10,"[['Adult'], ['Aged'], ['Biomarkers, Tumor', 'm...","[['D000328'], ['D000368'], ['D014408', 'Q00037...","[['G1000143'], ['MC_UU_12015/1'], ['MC_U106179...","[['Medical Research Council'], ['Medical Resea...",[nan],25637171
11,"[['Animals'], ['Birds', 'classification', 'gen...","[['D000818'], ['D001717', 'Q000145', 'Q000235'...","[['DP1 OD000448'], ['R01 HL087216']]","[['NIH HHS'], ['NHLBI NIH HHS']]","[['Avian genomes', 'Gene trees', 'Indels', 'Ph...",25741440
21,"[['Health Facilities'], ['Humans'], ['Malaria'...","[['D006268'], ['D006801'], ['D008288', 'Q00017...","[['U19 AI089674'], ['U19AI089674']]","[['NIAID NIH HHS'], ['NIAID NIH HHS']]",[nan],25366929


In [24]:
grant_funded['grant_ids_cleaned'] = grant_funded.grant_ids.apply(lambda x: re.sub("\[|'", '', x))
grant_funded['grant_agency_cleaned'] = grant_funded.grant_agency.apply(lambda x: re.sub("\[|'", '', x))
#re.sub("\[|\]|'", '', grant_funded.grant_ids.iloc[0]).split(',')

/home/riddleta/ac_knowl/ac_knowl/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/riddleta/ac_knowl/ac_knowl/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
grant_funded.head()

,mesh,mesh_id,grant_ids,grant_agency,keyword,pmid,grant_ids_cleaned,grant_agency_cleaned
3,[nan],[nan],"[['RC2 GM092561'], ['T32 GM083864']]","[['NIGMS NIH HHS'], ['NIGMS NIH HHS']]","[['gas chromatography', 'high performance liqu...",25789275,"RC2 GM092561], T32 GM083864]]","NIGMS NIH HHS], NIGMS NIH HHS]]"
6,[nan],[nan],"[['R01 DA007606'], ['R01 DA035499']]","[['NIDA NIH HHS'], ['NIDA NIH HHS']]","[['blood-brain barrier', 'glutamate', 'methamp...",25788874,"R01 DA007606], R01 DA035499]]","NIDA NIH HHS], NIDA NIH HHS]]"
10,"[['Adult'], ['Aged'], ['Biomarkers, Tumor', 'm...","[['D000328'], ['D000368'], ['D014408', 'Q00037...","[['G1000143'], ['MC_UU_12015/1'], ['MC_U106179...","[['Medical Research Council'], ['Medical Resea...",[nan],25637171,"G1000143], MC_UU_12015/1], MC_U106179471], 141...","Medical Research Council], Medical Research Co..."
11,"[['Animals'], ['Birds', 'classification', 'gen...","[['D000818'], ['D001717', 'Q000145', 'Q000235'...","[['DP1 OD000448'], ['R01 HL087216']]","[['NIH HHS'], ['NHLBI NIH HHS']]","[['Avian genomes', 'Gene trees', 'Indels', 'Ph...",25741440,"DP1 OD000448], R01 HL087216]]","NIH HHS], NHLBI NIH HHS]]"
21,"[['Health Facilities'], ['Humans'], ['Malaria'...","[['D006268'], ['D006801'], ['D008288', 'Q00017...","[['U19 AI089674'], ['U19AI089674']]","[['NIAID NIH HHS'], ['NIAID NIH HHS']]",[nan],25366929,"U19 AI089674], U19AI089674]]","NIAID NIH HHS], NIAID NIH HHS]]"


In [77]:
def interleave_lists(l1, l2):
    if len(l1)>1:
        outlist = ["{}---{}".format(b_, a_) for a_, b_ in zip(l1, l2)]
    else:
        outlist = [l1[0]+'---'+l2[0]]
        
    return(outlist)

In [78]:
grant_funded['grant_list'] = grant_funded.apply(lambda x: 
                                                interleave_lists(x['grant_ids_cleaned'].split('], '), 
                                                                 x['grant_agency_cleaned'].split('], ')), axis=1)

/home/riddleta/ac_knowl/ac_knowl/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [84]:
grant_pmids = grant_funded.explode('grant_list')
grant_pmids = grant_pmids[['pmid', 'grant_list']]

In [85]:
grant_pmids['grant_list'] = grant_pmids.grant_list.apply(lambda x: re.sub('\]', '', x))

In [87]:
grant_pmids[['agency', 'number', 'x1', 'x2']] = grant_pmids.grant_list.str.split('---', expand=True)

In [90]:
#nothing in x1 or x2 are from the NIH
grant_pmids = grant_pmids[grant_pmids.x1.isnull()]

In [111]:
grant_pmids.to_csv('output/pmcidextracted_grant_pmid_pairing.csv', index=False)

In [109]:
agency_counts = grant_pmids.agency.value_counts()
grant_id_counts = grant_pmids.number.value_counts()

In [103]:
agency_counts.iloc[0:10]

NCI NIH HHS      455167
NHLBI NIH HHS    325312
NIGMS NIH HHS    266381
NIAID NIH HHS    240204
NIDDK NIH HHS    206903
NIMH NIH HHS     177469
NIA NIH HHS      170175
NINDS NIH HHS    163838
NCRR NIH HHS     140881
NICHD NIH HHS    137069
Name: agency, dtype: int64

In [104]:
agency_counts.iloc[10:25]

NIDA NIH HHS                120229
Medical Research Council     87882
NIAMS NIH HHS                75422
NEI NIH HHS                  67507
NIEHS NIH HHS                66584
Wellcome Trust               64693
NCATS NIH HHS                59774
NIAAA NIH HHS                57386
NIBIB NIH HHS                56905
NIDCR NIH HHS                35811
NIDCD NIH HHS                35618
Intramural NIH HHS           31083
PHS HHS                      30654
NHGRI NIH HHS                26439
NIH HHS                      23841
Name: agency, dtype: int64

In [105]:
agency_counts.iloc[25:50]

Biotechnology and Biological Sciences Research Council                                 20526
                                                                                       18366
NIMHD NIH HHS                                                                          15317
NINR NIH HHS                                                                           14645
FIC NIH HHS                                                                            13409
Howard Hughes Medical Institute                                                        13356
Cancer Research UK                                                                     12814
British Heart Foundation                                                               12118
NCCIH NIH HHS                                                                          11891
Canadian Institutes of Health Research                                                  9749
WHI NIH HHS                                                           

In [112]:
75*100

7500

In [113]:
7500/175

42.857142857142854